In [ ]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
from math import exp
import ast
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

In [ ]:
# Initialize csv file paths
csv_file_path= 'merged_data_mod_scores_filtered_US_release_year_star_scores_prod_scores_dir_scores.csv'
# Create DataFrames from both CSV files
df = pd.read_csv(csv_file_path)

In [ ]:
#This function encodes MPAA 